In [2]:
import pandas as pd
from gensim.models import KeyedVectors

from data_iterator import LazyDataIterator

In [11]:
pd.concat([
    pd.read_csv("data/amazon_train.csv").fillna(""),
    pd.read_csv("data/amazon_val.csv").fillna("")
]).sample(frac=1).to_csv("data/amazon_trainval.csv", index=False)

In [12]:
pd.read_csv("data/amazon_trainval.csv")

,label,text
0,4,I had never seen book before but I found it ve...
1,1,This book is well written and includes helpful...
2,4,The Intelligent Investor is basically two book...
3,5,I love this book and have since before I could...
4,5,Incredible book and very informative. Books l...
...,...,...
3284995,5,"Good book, except no access code."
3284996,1,He did not appreciate it - threw it out and go...
3284997,3,I have read most of his books. This book was g...
3284998,5,great story well told


In [13]:
3_650_000 * 0.9

3285000.0